# Lab: Using a pre-trained model

We will use a pre-trained model to classify cats-and-dogs!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elephantscale/cool-ML-demos/blob/main/transfer-learning/transfer2-using-a-pre-trained-model.ipynb)

### Runtime
~ 30 minutes

### Note
Here we are dealing with real world images.  Processing them will required a lot of compute power.  
If you have access to, switch to **GPU** as run time!

### References
- https://www.tensorflow.org/tutorials/images/transfer_learning
- https://www.learnopencv.com/keras-tutorial-using-pre-trained-imagenet-models/

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

In [ ]:
## Loading our custom utils files
import os
import sys
from pathlib import Path

# Hack to download image utils when running on Colab ..etc
import os
import urllib.request

file_url = 'https://raw.githubusercontent.com/elephantscale/es-public/master/deep-learning/transfer_learning_utils.py'
file_location = "transfer_learning_utils.py"

if not os.path.exists (file_location):
    file_location = os.path.basename(file_location)
    if not os.path.exists(file_location):
        print("Downloading : ", file_url)
        urllib.request.urlretrieve(file_url, file_location)
# print('file_location:', file_location)

In [ ]:
## Loading our custom utils files
import os
import sys
from pathlib import Path

# Hack to download image utils when running on Colab ..etc
import os
import urllib.request

file_url = 'https://raw.githubusercontent.com/elephantscale/es-public/master/deep-learning/image_utils.py'
file_location = "image_utils.py"

if not os.path.exists (file_location):
    file_location = os.path.basename(file_location)
    if not os.path.exists(file_location):
        print("Downloading : ", file_url)
        urllib.request.urlretrieve(file_url, file_location)
# print('file_location:', file_location)

## TF-GPU Config
The following cell sets TF properties to run on GPU

In [ ]:
## This block is to tweak TF running on GPU
## You may comment this out, if you are not using GPU

## ---- start Memory setting ----
## Ask TF not to allocate all GPU memory at once.. allocate as needed
## Without this the execution will fail with "failed to initialize algorithm" error

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

## Step 1 - Download Data
We will use cat-dog-redux dataset

In [ ]:
## Common constants

IMG_WIDTH=160
IMG_HEIGHT=160
NUM_CLASSES=2
BATCH_SIZE=64

In [ ]:
import os

data_location = 'https://elephantscale-public.s3.amazonaws.com/data/images/cat-dog-redux.zip'

data_location_local = keras.utils.get_file(fname=os.path.basename(data_location),
                                           origin=data_location, extract=True)
print ('local download file: ', data_location_local)
data_dir = os.path.join(os.path.dirname(data_location_local), 'cat-dog-redux')
print ('local data dir: ', data_dir)
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'val')
print ('train dir:', train_dir)
print ('validation dir:', validation_dir)

## Step 2 - Download MobileNetv2 Model
Here we will examine MobileNetv2 model.  MobileNetv2 was developed at Google.  It is pre-trained on the ImageNet dataset consisting of 1.4 Million images and 1000 classes.

- paper: [MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)
- [github](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/README.md)
- [Tensorflow model page](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2)
- [Tensorflow transfer learning guide](https://www.tensorflow.org/tutorials/images/transfer_learning)

#### Notice the following:
- To use the MobileNet model, input image dimension should be one of (w,w,3)  
Image widths are one of 96, 128, 160, 192, 224(default).  
We are using 160 as that is closest to our images
- We are also initializing the model with the weights of 'imagenet'
- We are importing the full model (including the classifiying layer)


In [ ]:
from transfer_learning_utils import print_model_summary_compact

mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_WIDTH,IMG_HEIGHT,3), 
                                               include_top = True,
                                               weights = 'imagenet')
print_model_summary_compact(mobilenetv2_model)

# ! du -skh ~/.keras/models/*

## Step 3 - Predict on a Single Image

In [ ]:
def predict_on_single_image(image_file, model, model_pkg):
    from tensorflow.keras.preprocessing.image import load_img
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.preprocessing.image import img_to_array
    import os
    import random
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    
    print ("image_file:", image_file)
    img = image.load_img(image_file, target_size = (IMG_WIDTH, IMG_HEIGHT))
    # print ("image size: " , img.size)
    
    # scale the data
    img_data = image.img_to_array(img)
    # img_data = img_data / 255.0
    # print ('image_data.shape:', img_data.shape)
    
    ## The networks accept a 4-dimensional Tensor as an input of the form 
    ## ( batchsize, height, width, channels).
    img_data = np.expand_dims(img_data, axis = 0)
    # print('image_data converted to tensor form (batchsize, height, width, channels):', img_data.shape)
    
    ## Preprocess the input by subtracting the mean value from each channel of the images in the batch. 
    ## Mean is an array of three elements obtained by the average of R, G, B pixels of all images obtained from ImageNet. 
    ## The values for ImageNet are : [ 103.939, 116.779, 123.68 ]. 
    ## This is done using the preprocess_input() function.
    ## Using a copy here, because the function modifies the array
    processed_image = model_pkg.preprocess_input(img_data.copy())
    # print ('processed_image.shape:', processed_image.shape)

    ## Finally ready to prodict
    ## prediction will be a 1000 wide softmax array!
    ## Remember, ImageNet has 1000 classes
    prediction = model.predict (processed_image)
    # prediction = mobilenetv2_model.predict (img_data)
    # print ('prediction.shape:', prediction.shape)

    ## pull the highest probability from softmax output
    predictions2 = [ np.argmax(p) for p in prediction]
    index_of_highest_probability = np.argmax(prediction[0])
    value_of_highest_probability = prediction[0][index_of_highest_probability]
    print ("predicted class: ", index_of_highest_probability, ", probability:", value_of_highest_probability)
    
    
    ## This tells us a little about prediction
    decoded_predictions = model_pkg.decode_predictions(prediction)

    ## Here we are using a pandas dataframe to pretty-print top-5
    print ("Top-5 predictions:")
    df = pd.DataFrame(columns=['Class Id', 'Description', 'Probability (high to low)'])
    for i, p in enumerate(decoded_predictions[0]):
        # print ("    ", i, p)
        df.loc[i] = (p[0], p[1], p[2])

    ## plot the image
    # plt.figure()  # stack images vertically
    print(df)
    plt.imshow(img)
    plt.show()


## End: predict_on_single_image

In [ ]:
import random
from image_utils import get_image_files_from_dir

image_files = get_image_files_from_dir(validation_dir, recursive=True)
# print (image_files)

for i in range(5):
    # get a random image
    index = random.randint(0, len(image_files)-1)
    image_file = image_files[index]
    predict_on_single_image(image_file, mobilenetv2_model, tf.keras.applications.mobilenet)
    print ('============================================================================')
    

## ===================================================

## Step 4 - Inception Model

In [ ]:
IMG_WIDTH=299
IMG_HEIGHT=299

from transfer_learning_utils import print_model_summary_compact

inceptionv3_model = tf.keras.applications.InceptionV3(input_shape=(IMG_WIDTH,IMG_HEIGHT,3), 
                                               include_top = True,
                                               weights = 'imagenet')
print_model_summary_compact(inceptionv3_model)

# ! du -skh ~/.keras/models/*

In [ ]:
import random


image_files = get_image_files_from_dir(validation_dir, recursive=True)
# print (image_files)

for i in range(5):
    # get a random image
    index = random.randint(0, len(image_files)-1)
    image_file = image_files[index]
    predict_on_single_image(image_file, inceptionv3_model, tf.keras.applications.inception_v3)
    print ('============================================================================')
    

## Step 5 - Resnet Model

In [ ]:
IMG_WIDTH=224
IMG_HEIGHT=224

from transfer_learning_utils import print_model_summary_compact

resnet152v2_model = tf.keras.applications.ResNet152V2(input_shape=(IMG_WIDTH,IMG_HEIGHT,3), 
                                               include_top = True,
                                               weights = 'imagenet')
print_model_summary_compact(resnet152v2_model)

In [ ]:
import random


image_files = get_image_files_from_dir(validation_dir, recursive=True)
# print (image_files)

for i in range(5):
    # get a random image
    index = random.randint(0, len(image_files)-1)
    image_file = image_files[index]
    predict_on_single_image(image_file, resnet152v2_model, tf.keras.applications.resnet_v2)
    print ('============================================================================')
    